### Imports

In [9]:
import random
import pandas as pd

### Load data files

In [71]:
train_human_labels_trainable_top_10 = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable_top_10.csv')
train_human_labels_trainable_top_100 = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable_top_100.csv')
print(train_human_labels_trainable_top_10.head())

            ImageID        Source  LabelName  Confidence
0  000002b66c9c498e  verification  /m/01g317           1
1  000002b66c9c498e  verification   /m/09j2d           1
2  000002b66c9c498e  verification   /m/0dzct           1
3  000002b97e5471a0  verification   /m/0cgh4           1
4  000002c707c9895e  verification  /m/01g317           1


### Sort lables frequency

In [72]:
labels_count = train_human_labels_trainable_top_100['LabelName'].value_counts()
print(labels_count.head())

/m/01g317    807090
/m/09j2d     610840
/m/0dzct     331942
/m/07j7r     315026
/m/05s2s     266978
Name: LabelName, dtype: int64


### Merge labels with labels_count

In [73]:
train_human_labels_trainable_top_10 = train_human_labels_trainable_top_10.merge(labels_count.to_frame(), left_on='LabelName', right_index=True, how='left')
train_human_labels_trainable_top_10.rename(columns={'LabelName_x': 'LabelName', 'LabelName_y': 'freq'}, inplace=True)
train_human_labels_trainable_top_10.head()

,ImageID,Source,LabelName,Confidence,freq
0,000002b66c9c498e,verification,/m/01g317,1,807090
1,000002b66c9c498e,verification,/m/09j2d,1,610840
2,000002b66c9c498e,verification,/m/0dzct,1,331942
3,000002b97e5471a0,verification,/m/0cgh4,1,183125
4,000002c707c9895e,verification,/m/01g317,1,807090


### Group labels by image id

In [74]:
top_10_grouped = train_human_labels_trainable_top_10.groupby(['ImageID'])
top_100_grouped = train_human_labels_trainable_top_100.groupby(['ImageID'])

### Split defs

In [75]:
val_images_count = 10000

### Split top 10

In [76]:
top_10_images = list(top_10_grouped.groups.keys())
print('len(top_10_images) = {}'.format(len(top_10_images)))
random.shuffle(top_10_images)
top_10_images_train = set(top_10_images[:-val_images_count])
top_10_images_val = set(top_10_images[-val_images_count:])
print('len(top_10_images_train) = {}'.format(len(top_10_images_train)))
print('len(top_10_images_val) = {}'.format(len(top_10_images_val)))

len(top_10_images) = 1376466


len(top_10_images_train) = 1366466
len(top_10_images_val) = 10000


In [88]:
with open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_10_images_train_single.txt', 'w') as f_train,\
    open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_10_images_val_single.txt', 'w') as f_val:
    for name, group in top_10_grouped:
        f = f_val if name in top_10_images_val else f_train
        first_row = group.sort_values(by=['freq'], ascending=False).head(1)
        f.write('{} {}\n'.format(name, first_row['LabelName'].values[0]))